In [1]:
from sklearn.decomposition import PCA
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

import pandas as pd
import traceback
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

from joblib import dump, load
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
def read_UTK(startpath):
    X = []
    y = []
    try:
        for root, dirs, files in os.walk(startpath):
            for file in files:
                if file[-3:] == 'jpg':
                    names = file.split('_')
                    if int(names[0]) > 17 and int(names[0]) < 30:

                        y.append(names[1])
                        img = cv2.imread(root + file, cv2.IMREAD_GRAYSCALE)
                        X.append(img)

    except Exception as err:
        traceback.print_tb(err.__traceback__)
    return np.array(X, dtype='float'), np.array(y, dtype='uint8')

In [3]:
UTK_path = '../Data/UTKFace/'
X, y = read_UTK(UTK_path)
y_translation = {
    0: 'Male',
    1: 'Female',
}

In [4]:
X = np.expand_dims(X, -1)
oht = OneHotEncoder(categories='auto', sparse=False)
y = oht.fit_transform(y.reshape(-1, 1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
classes = 2
inputShape = (200, 200, 1)

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=inputShape, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# model.fit(X_train, y_train, batch_size=200, epochs=12)

In [9]:
model = load_model('gender_classifier_model')
# model.save('gender_classifier_model')

Instructions for updating:
Use tf.cast instead.


In [10]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Testing loss: {}, acc: {}\n'.format(loss, acc))

Testing loss: 0.08589147784392255, acc: 0.9719626168224299



In [11]:
pca = PCA(100)
X_train_reduced_100 = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_reduced_100 = pca.transform(X_test.reshape(X_test.shape[0], -1))

pca = PCA(50)
X_train_reduced_50 = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_reduced_50 = pca.transform(X_test.reshape(X_test.shape[0], -1))

pca = PCA(20)
X_train_reduced_20 = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_reduced_20 = pca.transform(X_test.reshape(X_test.shape[0], -1))

In [15]:
svc = SVC()
logreg = LogisticRegressionCV(max_iter=200)
nb = BernoulliNB()

In [13]:
svc.fit(X_train_reduced_20, np.argmax(y_train, axis=1))
svc.score(X_test_reduced_20, np.argmax(y_test, axis=1))

/Users/visheshhemnani/anaconda3/envs/ml_proj/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

0.612668743509865

In [16]:
logreg.fit(X_train_reduced_20, np.argmax(y_train, axis=1))
logreg.score(X_test_reduced_20, np.argmax(y_test, axis=1))

/Users/visheshhemnani/anaconda3/envs/ml_proj/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=200,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

0.7632398753894081

In [17]:
nb.fit(X_train_reduced_20, np.argmax(y_train, axis=1))
nb.score(X_test_reduced_20, np.argmax(y_test, axis=1))

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

0.7227414330218068

In [ ]:
running grid search on a smaller sampled size

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall', 'accuracy']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train_reduced_20, np.argmax(y_train, axis=1))

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = np.argmax(y_test, axis=1), clf.predict(X_test_reduced_20)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

# Tuning hyper-parameters for precision

